<a href="https://www.kaggle.com/code/elisamens/llm-sd-create-embeddings-quantify-similarity?scriptVersionId=136958268" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Code to:

Mens, E. C. (2023). Assessing Item Redundancy in Psychological Surveys by Using Large Language Models based Similarity Detection (LLM-SD) [Unpublished Masterthesis]. University of Amsterdam.

1. Embeddings by Ada
2. Embeddings by sBERT
3. Embeddings by USE
4. Average results
5. Identify if both items are belonging to the same scale and the same construct
6. Interactive plot based on e.g. Ada embedding
7. Link to R code for Networkplots 

It is not nesceccary to run the embeddingsmodels before viewing the results. 
Output data from previous notebooks is included. 

In [ ]:
!pip install openai --root-user-action=ignore
!pip install -U sentence-transformers --root-user-action=ignore
!pip install researchpy --root-user-action=ignore

## Import libraries

In [4]:
import pandas as pd
import numpy as np
import openai
from openai.embeddings_utils import get_embedding #Open Ai embeddings
from sentence_transformers import SentenceTransformer #Sbert embeddings
from numpy.linalg import norm # used for cosine function 
import tensorflow_hub as hub #USE embeddings
import tensorflow as tf #USE embeddings
import tensorflow_datasets as tfds #USE embeddings


## Import data

In [ ]:
path = "/kaggle/input/constructs-items/"
items = pd.read_excel('/kaggle/input/items-original/Constructs_items.xlsx')
items = items.loc[0:319, ('Type of construct', 'Construct', 'Measure', 'Item_English', 'Answer type')]

# Selecting relevant columns, and remove empty rows
df = items[['Type of construct', 'Construct', 'Measure', 'Item_English']]

# Converting into strings & removing line breaks 
df.loc[:,("Item_English")] = df.loc[:,("Item_English")].astype(str).apply(lambda x: ' '.join(x.split('\n')))

## The Models

### 1. Ada Embeddings Open Ai

#### Model setup

In [ ]:
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
openai.api_key = x # personal API key
embedding_name = "ada_embeddings"

#### Embeddings

In [ ]:
df[embedding_name] = df["Item_English"].apply(lambda x: get_embedding(x, engine = embedding_model))
df.to_csv(embedding_name + '.csv')

#### Cosine Similarity Matrix

In [ ]:
#defining cosine function
def cosine(x, y):
    cosine = np.dot(x, y) / (norm(x) * norm(y))
    return cosine

#setting up df for cosine based on OpenAi
CSI_ada = df[['Item_English', embedding_name]]

for i in range(len(CSI_ada)) :
    x = CSI_ada.iloc[i,1]
    z = CSI_ada.apply(lambda row: (cosine(row['ada_embeddings'], x)), axis=1)
    col_name = str(i) + '. ' + str(CSI_ada.iloc[i,0])
    CSI_ada.loc[:, col_name] = z

CSI_ada.to_csv("CSI_ada.csv")    
print("\n---- Output: cosine similarity of items by ada ----\n")


#### Cosine Similarity Ranking of Items 

In [ ]:
ranking_ada = CSI_ada

# Drop embedding column
ranking_ada = ranking_ada.drop(['ada_embeddings'], axis=1)

# Create Long format
ranking_ada = pd.DataFrame({ 'Item_2': np.tile(ranking_ada.columns, len(ranking_ada)), 'Item_1': ranking_ada.Item_English.repeat(ranking_ada.shape[1]), 'CSI': ranking_ada.values.ravel()})
ranking_ada = ranking_ada[ranking_ada['Item_2'] != 'Item_English']

#Adding index columns
ranking_ada['Index_1'] = ranking_ada.index
ranking_ada['Index_2'], ranking_ada['Item_2'] = ranking_ada['Item_2'].str.split('.', 1).str
ranking_ada['Index_2'] = ranking_ada['Index_2'].astype(int)

# Rearrange column and sort values 
ranking_ada = ranking_ada[["Index_1","Item_1","Index_2",'Item_2', 'CSI']]
ranking_ada = ranking_ada.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
ranking_ada = ranking_ada[ranking_ada['Index_1'] != ranking_ada['Index_2']]
ranking_ada.reset_index(inplace=True, drop=True)

#Save data
ranking_ada.to_csv('ranking_ada_duplicates.csv', index=False)
#ranking_ada.to_excel('ranking_ada_duplicates.xlsx') 

print("\n----Output: ranking by ada with duplicates in Long Format ----\n")

# removing dupclite items pairs e.g. 35-36 & 36-35
ranking_ada = ranking_ada[['Index_1', 'Item_1', 'Index_2', 'Item_2', 'CSI']][::2]
ranking_ada.reset_index(inplace=True, drop=True)

#Save data
ranking_ada.to_csv('ranking_ada.csv', index=False)
#ranking_ada.to_excel('ranking_ada.xlsx') 

print("\n----Output: ranking by ada in long format----\n")


#### Average Embeddings per Construct Type, Construct and Measurement

In [ ]:
test = df
#split embedding over multiple columns and concatenate them with original columns
test = pd.concat([test, test["ada_embeddings"].apply(pd.Series)], axis = 1)

#average embedding vector per ...
constructtype_ada_embeddings = test.groupby("Type of construct").mean()
constructtype_ada_embeddings.reset_index(inplace=True)
constructtype_ada_embeddings.to_csv("constructtype_ada_embeddings.csv") 
print("\n---- Output: Average Embeddings of Construct Type by ada ----\n")


construct_ada_embeddings = test.groupby("Construct").mean()
construct_ada_embeddings.reset_index(inplace=True)
construct_ada_embeddings.to_csv("construct_ada_embeddings.csv") 
print("\n---- Output: Average Embeddings of Construct by ada ----\n")


measurement_ada_embeddings = test.groupby("Measure").mean()
measurement_ada_embeddings.reset_index(inplace=True)
measurement_ada_embeddings.to_csv("measurement_ada_embeddings.csv") 
print("\n---- Output: Average Embeddings of Measurement by ada ----\n")


# cosine similarity between Type of construct
constructtype_ada_CSI = constructtype_ada_embeddings[['Type of construct']]
del constructtype_ada_embeddings['Type of construct']

for i in range(len(constructtype_ada_embeddings)) :
    x = constructtype_ada_embeddings.iloc[i,:]
    z = constructtype_ada_embeddings.apply(lambda row: (cosine(row, x)), axis=1)
    col_name = str(constructtype_ada_CSI.iloc[i,0])
    constructtype_ada_CSI.loc[:, col_name] = z
constructtype_ada_CSI.to_csv("constructtype_ada_CSI.csv")    
print("\n---- Output: Cosine Similiarity Construct Type by ada ----\n")

# cosine similarity between construct
construct_ada_CSI = construct_ada_embeddings[['Construct']]
del construct_ada_embeddings['Construct']

for i in range(len(construct_ada_embeddings)) :
    x = construct_ada_embeddings.iloc[i,:]
    z = construct_ada_embeddings.apply(lambda row: (cosine(row, x)), axis=1)
    col_name = str(construct_ada_CSI.iloc[i,0])
    construct_ada_CSI.loc[:, col_name] = z
construct_ada_CSI.to_csv("construct_ada_CSI.csv") 
print("\n---- Output: Cosine Similiarity Construct by ada ----\n")


# cosine similarity between measurement
measurement_ada_CSI = measurement_ada_embeddings[['Measure']]
del measurement_ada_embeddings['Measure']

for i in range(len(measurement_ada_embeddings)) :
    x = measurement_ada_embeddings.iloc[i,:]
    z = measurement_ada_embeddings.apply(lambda row: (cosine(row, x)), axis=1)
    col_name = str(measurement_ada_CSI.iloc[i,0])
    measurement_ada_CSI.loc[:, col_name] = z
measurement_ada_CSI.to_csv("measurement_ada_CSI.csv") 
print("\n---- Output: Cosine Similiarity Measurement by ada ----\n")


#### Ranking by Ada for Construct Type, Construct and Measurement

In [ ]:
test = constructtype_ada_CSI

test = pd.DataFrame({ 'Constructtype_2': np.tile(test.columns, len(test)), 
                            'Constructtype_1': test['Type of construct'].repeat(test.shape[1]), 
                            'CSI': test.values.ravel()})
test = test[test['Constructtype_2'] != 'Type of construct']

# Rearrange column and sort values 
test = test[["Constructtype_1","Constructtype_2",'CSI']]
test = test.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
test = test[test['Constructtype_1'] != test['Constructtype_2']]
test.reset_index(inplace=True, drop=True)

# removing dupclite items pairs e.g. 35-36 & 36-35
#test = test[["Constructtype_1","Constructtype_2",'CSI']][::2]
#test.reset_index(inplace=True, drop=True)

#Save data
ranking_constructtype_ada = test.rename(columns={'CSI': 'ada'})
ranking_constructtype_ada.to_csv('ranking_constructtype_ada.csv', index=False)

print("\n----Output: ranking Construct type by ada in long format with duplicates----\n")



##############################################################################
test = construct_ada_CSI

test = pd.DataFrame({ 'Construct_2': np.tile(test.columns, len(test)), 
                            'Construct_1': test['Construct'].repeat(test.shape[1]), 
                            'CSI': test.values.ravel()})
test = test[test['Construct_2'] != 'Construct']

# Rearrange column and sort values 
test = test[["Construct_1","Construct_2",'CSI']]
test = test.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
test = test[test['Construct_1'] != test['Construct_2']]
test.reset_index(inplace=True, drop=True)

# removing dupclite items pairs e.g. 35-36 & 36-35
#test = test[["Construct_1","Construct_2",'CSI']][::2]
#test.reset_index(inplace=True, drop=True)

#Save data
ranking_construct_ada = test.rename(columns={'CSI' : 'ada'})
ranking_construct_ada.to_csv('ranking_construct_ada.csv', index=False)
#ranking_construct_ada.to_excel('ranking_construct_ada.xlsx') 

print("\n----Output: ranking Construct by ada in long format with dupcliates ----\n")



##############################################################################
test = measurement_ada_CSI

test = pd.DataFrame({ 'Measure_2': np.tile(test.columns, len(test)), 
                            'Measure_1': test['Measure'].repeat(test.shape[1]), 
                            'CSI': test.values.ravel()})
test = test[test['Measure_2'] != 'Measure']

# Rearrange column and sort values 
test = test[["Measure_1","Measure_2",'CSI']]
test = test.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
test = test[test['Measure_1'] != test['Measure_2']]
test.reset_index(inplace=True, drop=True)

# removing dupclite items pairs e.g. 35-36 & 36-35
#test = test[["Measure_1","Measure_2",'CSI']][::2]
#test.reset_index(inplace=True, drop=True)

#Save data
ranking_measurement_ada = test.rename(columns={'CSI' : 'ada'})
ranking_measurement_ada.to_csv('ranking_measurement_ada.csv', index=False)
#ranking_measurement_ada.to_excel('ranking_construct_ada.xlsx') 

print("\n----Output: ranking Measurement by ada in long format with duplicates----\n")


### 2. SBert embeddings Google

#### Model setup

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_name = "sBert_embeddings"

#### Embeddings

In [ ]:
df[embedding_name] = df["Item_English"].apply(lambda x: embedding_model.encode(x))

# Save output data
column_list=["Type of construct", "Construct", "Measure", "Item_English", "sBert_embeddings"]
df.to_csv(embedding_name + '.csv', columns = column_list, index=False)
#df.to_excel(embedding_name + '.xlsx', columns = column_list, index=False)  

#### Cosine Similarity Matrix

In [ ]:
minrange = -1
maxrange = 1

#setting up df for cosine based on sBert
CSI_sBert = df[['Item_English', 'sBert_embeddings']]

for i in range(len(CSI_sBert)) :
    x = CSI_sBert.iloc[i,1]
    z = CSI_sBert.apply(lambda row: (cosine(row['sBert_embeddings'], x)), axis=1)
    y = (z - minrange)/(maxrange-minrange) #Force the CSI to be within 0:1 range
    col_name = str(i) + '. ' + str(CSI_sBert.iloc[i,0])
    CSI_sBert.loc[:, col_name] = y

CSI_sBert.to_csv("CSI_sBert.csv")    
#CSI_sBert.to_excel('CSI_sBert.xlsx')  

print("\n----Output: Cosine Similariy by sBert ----\n")

#### Cosine Similarity Ranking 

In [ ]:
ranking_sBert = CSI_sBert

# Drop embedding column
ranking_sBert = ranking_sBert.drop(['sBert_embeddings'], axis=1)

# Create Long format
ranking_sBert = pd.DataFrame({ 'Item_2': np.tile(ranking_sBert.columns, len(ranking_sBert)), 'Item_1': ranking_sBert.Item_English.repeat(ranking_sBert.shape[1]), 'CSI': ranking_sBert.values.ravel()})
ranking_sBert = ranking_sBert[ranking_sBert['Item_2'] != 'Item_English']

#Adding index columns
ranking_sBert['Index_1'] = ranking_sBert.index
ranking_sBert['Index_2'], ranking_sBert['Item_2'] = ranking_sBert['Item_2'].str.split('.', 1).str
ranking_sBert['Index_2'] = ranking_sBert['Index_2'].astype(int)

# Rearrange column and sort values 
ranking_sBert = ranking_sBert[["Index_1","Item_1","Index_2",'Item_2', 'CSI']]
ranking_sBert = ranking_sBert.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
ranking_sBert = ranking_sBert[ranking_sBert['Index_1'] != ranking_sBert['Index_2']]
ranking_sBert.reset_index(inplace=True, drop=True)

#Save data without removing duplicates
ranking_sBert.to_csv('ranking_sBert_duplipcates.csv', index=False)
ranking_sBert.to_excel('ranking_sBert_duplicates.xlsx') 
print("\n----Output: Ranking by sBert with duplicates in Long Format----\n")


# removing dupclite items pairs e.g. 35-36 & 36-35
ranking_sBert = ranking_sBert[['Index_1', 'Item_1', 'Index_2', 'Item_2', 'CSI']][::2]
ranking_sBert.reset_index(inplace=True, drop=True)

#Save data
ranking_sBert.to_csv('ranking_sBert.csv', index=False)
#ranking_sBert.to_excel('ranking_sBert.xlsx') 

print("\n----Output: Ranking by sBert in Long Format ----\n")


### Average embeddings per Construct type Construct and Measurement

In [ ]:
test = df.loc[:,('Type of construct', 'Construct', 'Measure', 'Item_English', 'sBert_embeddings')]

#split embedding over multiple columns and concatenate them with original columns
test = pd.concat([test, test["sBert_embeddings"].apply(pd.Series)], axis = 1)

#average embedding vector per ...
constructtype_sbert_embeddings = test.groupby("Type of construct").mean()
constructtype_sbert_embeddings.reset_index(inplace=True)
constructtype_sbert_embeddings.to_csv("constructtype_sbert_embeddings.csv") 
print("\n----Output: Average Embeddings for Construct type by sBert ----\n")

construct_sbert_embeddings = test.groupby("Construct").mean()
construct_sbert_embeddings.reset_index(inplace=True)
construct_sbert_embeddings.to_csv("construct_sbert_embeddings.csv") 
print("\n----Output: Average Embeddings for Construct by sBert ----\n")

measurement_sbert_embeddings = test.groupby("Measure").mean()
measurement_sbert_embeddings.reset_index(inplace=True)
measurement_sbert_embeddings.to_csv("measurement_sbert_embeddings.csv") 
print("\n----Output: Average Embeddings for Measurementby sBert ----\n")


# cosine similarity between Type of construct
constructtype_sbert_CSI = constructtype_sbert_embeddings[['Type of construct']]
del constructtype_sbert_embeddings['Type of construct']

for i in range(len(constructtype_sbert_embeddings)) :
    x = constructtype_sbert_embeddings.iloc[i,:]
    z = constructtype_sbert_embeddings.apply(lambda row: (cosine(row, x)), axis=1)
    y = (z - minrange)/(maxrange-minrange)
    col_name = str(constructtype_sbert_CSI.iloc[i,0])
    constructtype_sbert_CSI.loc[:, col_name] = y
constructtype_sbert_CSI.to_csv("constructtype_sbert_CSI.csv")    
print("\n----Output: CSI between Construct type by sBert ----\n")


# cosine similarity between construct
construct_sbert_CSI = construct_sbert_embeddings[['Construct']]
del construct_sbert_embeddings['Construct']

for i in range(len(construct_sbert_embeddings)) :
    x = construct_sbert_embeddings.iloc[i,:]
    z = construct_sbert_embeddings.apply(lambda row: (cosine(row, x)), axis=1)
    y = (z - minrange)/(maxrange-minrange)
    col_name = str(construct_sbert_CSI.iloc[i,0])
    construct_sbert_CSI.loc[:, col_name] = y
construct_sbert_CSI.to_csv("construct_sbert_CSI.csv") 
print("\n----Output: CSI between Construct  by sBert ----\n")

# cosine similarity between measurement
measurement_sbert_CSI = measurement_sbert_embeddings[['Measure']]
del measurement_sbert_embeddings['Measure']

for i in range(len(measurement_sbert_embeddings)) :
    x = measurement_sbert_embeddings.iloc[i,:]
    z = measurement_sbert_embeddings.apply(lambda row: (cosine(row, x)), axis=1)
    y = (z - minrange)/(maxrange-minrange)
    col_name = str(measurement_sbert_CSI.iloc[i,0])
    measurement_sbert_CSI.loc[:, col_name] = y
measurement_sbert_CSI.to_csv("measurement_sbert_CSI.csv") 
print("\n----Output: CSI between Measurement by sBert ----\n")



### Cosine Similarity Between Construct type, Construct and Measurement

In [ ]:
test = constructtype_sbert_CSI

test = pd.DataFrame({ 'Constructtype_2': np.tile(test.columns, len(test)), 
                            'Constructtype_1': test['Type of construct'].repeat(test.shape[1]), 
                            'CSI': test.values.ravel()})
test = test[test['Constructtype_2'] != 'Type of construct']

# Rearrange column and sort values 
test = test[["Constructtype_1","Constructtype_2",'CSI']]
test = test.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
test = test[test['Constructtype_1'] != test['Constructtype_2']]
test.reset_index(inplace=True, drop=True)

# removing dupclite items pairs e.g. 35-36 & 36-35
#test = test[["Constructtype_1","Constructtype_2",'CSI']][::2]
#test.reset_index(inplace=True, drop=True)

#Save data
ranking_constructtype_sbert = test.rename(columns={'CSI': 'sbert'})
ranking_constructtype_sbert.to_csv('ranking_constructtype_sbert.csv', index=False)
#ranking_constructtype_sbert.to_excel('ranking_constructtype_sbert.xlsx') 

print("\n----Output: ranking Construct type by sbert in long format with duplicates----\n")

##############################################################################

test = construct_sbert_CSI

test = pd.DataFrame({ 'Construct_2': np.tile(test.columns, len(test)), 
                            'Construct_1': test['Construct'].repeat(test.shape[1]), 
                            'CSI': test.values.ravel()})
test = test[test['Construct_2'] != 'Construct']

# Rearrange column and sort values 
test = test[["Construct_1","Construct_2",'CSI']]
test = test.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
test = test[test['Construct_1'] != test['Construct_2']]
test.reset_index(inplace=True, drop=True)

# removing dupclite items pairs e.g. 35-36 & 36-35
#test = test[["Construct_1","Construct_2",'CSI']][::2]
#test.reset_index(inplace=True, drop=True)

#Save data
ranking_construct_sbert = test.rename(columns={'CSI' : 'sbert'})
ranking_construct_sbert.to_csv('ranking_construct_sbert.csv', index=False)
#ranking_construct_sbert.to_excel('ranking_construct_sbert.xlsx') 

print("\n----Output: ranking Construct by sbert in long format with dupcliates----\n")

##############################################################################

test = measurement_sbert_CSI

test = pd.DataFrame({ 'Measure_2': np.tile(test.columns, len(test)), 
                            'Measure_1': test['Measure'].repeat(test.shape[1]), 
                            'CSI': test.values.ravel()})
test = test[test['Measure_2'] != 'Measure']

# Rearrange column and sort values 
test = test[["Measure_1","Measure_2",'CSI']]
test = test.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
test = test[test['Measure_1'] != test['Measure_2']]
test.reset_index(inplace=True, drop=True)

# removing dupclite items pairs e.g. 35-36 & 36-35
#test = test[["Measure_1","Measure_2",'CSI']][::2]
#test.reset_index(inplace=True, drop=True)

#Save data
ranking_measurement_sbert = test.rename(columns={'CSI' : 'sbert'})
ranking_measurement_sbert.to_csv('ranking_measurement_sbert.csv', index=False)
#ranking_measurement_sbert.to_excel('ranking_measurement_sbert.xlsx') 

print("\n----Output: ranking Measurement by sbert in long format with duplicates----\n")


### 3. Universal Sentence Encoder Google

#### Model setup

In [ ]:
# Load pre-trained universal sentence encoder model
embedding_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embedding_name = "USE_embeddings"


#### Embeddings

In [ ]:
x = embedding_model(df["Item_English"])
#print(len(df["Item_English"]))
#print(len(x))
#print(df["Item_English"].shape)
#print(x.shape)

# Tenserflow to panda dataframe 
y = x.numpy()
h = pd.DataFrame(y)
h.shape


#cosine(h.loc[269,0:511], h.loc[269, 0:511])
#Combine 512 dimensions into one column
df[embedding_name] = h.astype(str).agg(', '.join, axis=1)
df[embedding_name] = h.astype(str).agg(', '.join, axis=1)

# Save output data
column_list=["Type of construct", "Construct", "Measure", "Item_English", "USE_embeddings"]
df.to_csv(embedding_name + '.csv', columns = column_list, index=False)
#df.to_excel(embedding_name + '.xlsx', columns = column_list, index=False)  

#### Cosine Similarity Matrix

In [ ]:
#setting up df for cosine based on OpenAi
CSI_USE_copy = pd.concat([df['Item_English'], h], axis=1)
CSI_USE = pd.DataFrame(df['Item_English'])

#calculate cosine and setting up df
for i in range(len(CSI_USE)) :
    x = CSI_USE_copy.iloc[i,1:512]
    z = CSI_USE_copy.apply(lambda row: (cosine(row[1:512], x)), axis=1)
    y = (z - minrange)/(maxrange-minrange)
    col_name = str(i) + '. ' + str(CSI_USE.iloc[i,0])
    CSI_USE.loc[:, col_name] = y
    
CSI_USE.to_csv("CSI_USE.csv")  
#CSI_USE.to_excel('CSI_USE.xlsx')  

print("\n----Output: Cosine Similarity by USE ----\n")


#### Cosine Similarity Ranking 

In [ ]:
ranking_USE = CSI_USE

# Drop embedding column
#ranking_USE = ranking_USE.drop(['sBert_embeddings'], axis=1)

# Create Long format
ranking_USE = pd.DataFrame({ 'Item_2': np.tile(ranking_USE.columns, len(ranking_USE)), 'Item_1': ranking_USE.Item_English.repeat(ranking_USE.shape[1]), 'CSI': ranking_USE.values.ravel()})
ranking_USE = ranking_USE[ranking_USE['Item_2'] != 'Item_English']

#Adding index columns
ranking_USE['Index_1'] = ranking_USE.index
ranking_USE['Index_2'], ranking_USE['Item_2'] = ranking_USE['Item_2'].str.split('.', 1).str
ranking_USE['Index_2'] = ranking_USE['Index_2'].astype(int)

# Rearrange column and sort values 
ranking_USE = ranking_USE[["Index_1","Item_1","Index_2",'Item_2', 'CSI']]
ranking_USE = ranking_USE.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
ranking_USE = ranking_USE[ranking_USE['Index_1'] != ranking_USE['Index_2']]
ranking_USE.reset_index(inplace=True, drop=True)

#Save data
ranking_USE.to_csv('ranking_USE_duplicates.csv', index=False)
#ranking_USE.to_excel('ranking_USE_duplicates.xlsx') 
print("\n----Output: ranking by USE with duplicates in Long Format----\n")


# removing dupclite items pairs e.g. 35-36 & 36-35
ranking_USE = ranking_USE[['Index_1', 'Item_1', 'Index_2', 'Item_2', 'CSI']][::2]
ranking_USE.reset_index(inplace=True, drop=True)

#Save data
ranking_USE.to_csv('ranking_USE.csv', index=False)
#ranking_USE.to_excel('ranking_USE.xlsx') 

print("\n----Output: ranking by USE in Long Format----\n")


### Averages Embedding per construct type construct and measurment

In [ ]:
test = df.loc[:,('Type of construct', 'Construct', 'Measure', 'Item_English')]
#split embedding over multiple columns and concatenate them with original columns
test = pd.concat([test, h], axis = 1)

#average embedding vector per ...
constructtype_USE_embeddings = test.groupby("Type of construct").mean()
constructtype_USE_embeddings.reset_index(inplace=True)
constructtype_USE_embeddings.to_csv("constructtype_USE_embeddings.csv") 
print("\n----Output: Average embeddings per COnstruct type by USE----\n")

construct_USE_embeddings = test.groupby("Construct").mean()
construct_USE_embeddings.reset_index(inplace=True)
construct_USE_embeddings.to_csv("construct_USE_embeddings.csv") 
print("\n----Output: Average embeddings per COnstruct by USE----\n")


measurement_USE_embeddings = test.groupby("Measure").mean()
measurement_USE_embeddings.reset_index(inplace=True)
measurement_USE_embeddings.to_csv("measurement_USE_embeddings.csv") 
print("\n----Output: Average embeddings per Measurement by USE----\n")


# cosine similarity between Type of construct
constructtype_USE_CSI = constructtype_USE_embeddings[['Type of construct']]
del constructtype_USE_embeddings['Type of construct']

for i in range(len(constructtype_USE_embeddings)) :
    x = constructtype_USE_embeddings.iloc[i,:]
    z = constructtype_USE_embeddings.apply(lambda row: (cosine(row, x)), axis=1)
    y = (z - minrange)/(maxrange-minrange)
    col_name = str(constructtype_USE_CSI.iloc[i,0])
    constructtype_USE_CSI.loc[:, col_name] = y
constructtype_USE_CSI.to_csv("constructtype_USE_CSI.csv")    
print("\n----Output: Cosine Similarity per Construct type by USE----\n")

# cosine similarity between construct
construct_USE_CSI = construct_USE_embeddings[['Construct']]
del construct_USE_embeddings['Construct']

for i in range(len(construct_USE_embeddings)) :
    x = construct_USE_embeddings.iloc[i,:]
    z = construct_USE_embeddings.apply(lambda row: (cosine(row, x)), axis=1)
    y = (z - minrange)/(maxrange-minrange)
    col_name = str(construct_USE_CSI.iloc[i,0])
    construct_USE_CSI.loc[:, col_name] = y
construct_USE_CSI.to_csv("construct_USE_CSI.csv") 
print("\n----Output: Cosine Similarity per Construct by USE----\n")


# cosine similarity between measurement
measurement_USE_CSI = measurement_USE_embeddings[['Measure']]
del measurement_USE_embeddings['Measure']

for i in range(len(measurement_USE_embeddings)) :
    x = measurement_USE_embeddings.iloc[i,:]
    z = measurement_USE_embeddings.apply(lambda row: (cosine(row, x)), axis=1)
    y = (z - minrange)/(maxrange-minrange)
    col_name = str(measurement_USE_CSI.iloc[i,0])
    measurement_USE_CSI.loc[:, col_name] = y
measurement_USE_CSI.to_csv("measurement_USE_CSI.csv") 
print("\n----Output: Cosine Similarity per Measurement by USE----\n")



In [ ]:
test = constructtype_USE_CSI

test = pd.DataFrame({ 'Constructtype_2': np.tile(test.columns, len(test)), 
                            'Constructtype_1': test['Type of construct'].repeat(test.shape[1]), 
                            'CSI': test.values.ravel()})
test = test[test['Constructtype_2'] != 'Type of construct']

# Rearrange column and sort values 
test = test[["Constructtype_1","Constructtype_2",'CSI']]
test = test.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
test = test[test['Constructtype_1'] != test['Constructtype_2']]
test.reset_index(inplace=True, drop=True)

# removing dupclite items pairs e.g. 35-36 & 36-35
#test = test[["Constructtype_1","Constructtype_2",'CSI']][::2]
#test.reset_index(inplace=True, drop=True)

#Save data
ranking_constructtype_USE = test.rename(columns={'CSI': 'USE'})
ranking_constructtype_USE.to_csv('ranking_constructtype_USE.csv', index=False)
#ranking_constructtype_USE.to_excel('ranking_constructtype_USE.xlsx') 

print("\n----Output: ranking Construct type by USE in long format with duplicates----\n")

##############################################################################

test = construct_USE_CSI

test = pd.DataFrame({ 'Construct_2': np.tile(test.columns, len(test)), 
                            'Construct_1': test['Construct'].repeat(test.shape[1]), 
                            'CSI': test.values.ravel()})
test = test[test['Construct_2'] != 'Construct']

# Rearrange column and sort values 
test = test[["Construct_1","Construct_2",'CSI']]
test = test.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
test = test[test['Construct_1'] != test['Construct_2']]
test.reset_index(inplace=True, drop=True)

# removing dupclite items pairs e.g. 35-36 & 36-35
#test = test[["Construct_1","Construct_2",'CSI']][::2]
#test.reset_index(inplace=True, drop=True)

#Save data
ranking_construct_USE = test.rename(columns={'CSI' : 'USE'})
ranking_construct_USE.to_csv('ranking_construct_USE.csv', index=False)
#ranking_construct_USE.to_excel('ranking_construct_USE.xlsx') 

##############################################################################

print("\n----Output: ranking Construct by USE in long format with dupcliates----\n")

test = measurement_USE_CSI

test = pd.DataFrame({ 'Measure_2': np.tile(test.columns, len(test)), 
                            'Measure_1': test['Measure'].repeat(test.shape[1]), 
                            'CSI': test.values.ravel()})
test = test[test['Measure_2'] != 'Measure']

# Rearrange column and sort values 
test = test[["Measure_1","Measure_2",'CSI']]
test = test.sort_values(by = ['CSI'], ascending = False, ignore_index=True)

# Drop row if items are the same 
test = test[test['Measure_1'] != test['Measure_2']]
test.reset_index(inplace=True, drop=True)

# removing dupclite items pairs e.g. 35-36 & 36-35
#test = test[["Measure_1","Measure_2",'CSI']][::2]
#test.reset_index(inplace=True, drop=True)

#Save data
ranking_measurement_USE = test.rename(columns={'CSI' : 'USE'})
ranking_measurement_USE.to_csv('ranking_measurement_USE.csv', index=False)
#ranking_measurement_USE.to_excel('ranking_measurement_USE.xlsx') 

print("\n----Output: ranking Measurement by USE in long format with dupcliates----\n")


### 4. Average Results

In [ ]:
# Import data without running the models
path = '/kaggle/input/llm-sd/'
ada = pd.read_csv(path + 'ranking_ada_duplicates.csv').rename(columns = {'CSI' : 'ada'})
sbert = pd.read_csv(path + 'ranking_sBert_duplipcates.csv').rename(columns = {'CSI' : 'sbert'})
use = pd.read_csv(path + 'ranking_USE_duplicates.csv').rename(columns = {'CSI' : 'USE'})

df = pd.merge(ada, sbert,  how='left', left_on=['Index_1','Item_1', 'Index_2', 'Item_2'], right_on = ['Index_1','Item_1', 'Index_2', 'Item_2'])
df = pd.merge(df, use,  how='left', left_on=['Index_1','Item_1', 'Index_2', 'Item_2'], right_on = ['Index_1','Item_1', 'Index_2', 'Item_2'])

# removing dupclite items pairs e.g. 35-36 & 36-35
df = df[['Index_1', 'Item_1', 'Index_2', 'Item_2', 'ada', 'sbert', 'USE']][::2]
df.reset_index(inplace=True, drop=True)

df['csi_mean'] = df.loc[:,('ada', 'sbert', 'USE')].mean(axis=1)

df = df.drop(df.loc[:,('ada', 'sbert', 'USE')], axis=1)


### 5. Identify if both items in the pair from the same scale and the same construct

In [ ]:
embeddings_df = pd.read_csv('/kaggle/input/original-df/df.csv')
copy_df = embeddings_df.loc[:,['Construct', 'Measure', 'Item_English']].rename(columns = {'Item_English' : 'Item_1'})
copy_df['Index'] = copy_df.index.astype(int)
df = pd.merge(df, copy_df.loc[:,['Construct', 'Measure', 'Index']],  how='left', left_on = ['Index_1'], right_on = ['Index'])
df = pd.merge(df, copy_df.loc[:,['Construct', 'Measure', 'Index']],  how='left', left_on = ['Index_2'], right_on = ['Index'])

# Pair from the same measurement, yes = 1, no = 0
df['same_measurement'] = np.where((df['Measure_x'] == df['Measure_y']), 1, 0)
# Pair from the same construct, yes = 1, no = 0
df['same_construct'] = np.where((df['Construct_x'] == df['Construct_y']), 1, 0)

df = df.drop(df.loc[:,('Construct_x','Measure_x','Index_x','Construct_y','Measure_y','Index_y')], axis=1)

### 6. Interactive plot based on e.g. Ada embedding

In [10]:
from sklearn.manifold import TSNE 
import matplotlib.pyplot as plt
import plotly.express as px

plot_df = pd.read_csv('/kaggle/input/ada-embeddings/ada_embeddings.csv')
del plot_df[plot_df.columns[0]]
matrix = np.array(plot_df.ada_embeddings.apply(eval).to_list())

tsne = TSNE(n_components=2, perplexity=15, random_state=42, init='random', learning_rate=200)
vis_dims = tsne.fit_transform(matrix)
plot_df["embed_vis2d"] = vis_dims.tolist()
plot_df.columns
vis_dims = pd.DataFrame(vis_dims).rename(columns={0:'x',1:'y'})
categories = plot_df["Construct"].unique()

fig = px.scatter(
    vis_dims, x='x', y='y',
    color=plot_df["Construct"],
    color_discrete_sequence=px.colors.qualitative.Alphabet,
    hover_name=plot_df["Item_English"],
    size_max=5, opacity=0.6, template='simple_white',
    title='Ada Embeddings of Items',
    width=900, height=900)
#fig.update_layout(legend_title_text="Construct")
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(showlegend=False)
fig.show()

### 7. Link to R code for Networkplots 

https://github.com/ECMens/LLM-SD-to-Detect-Item-Redundancy-in-Psychological-Surveys.git
- Item_Network_Code.Rmd: Rmarkdown file with code to create network plot
- Item_Network_Code.html